Nama : Najla Dhia Rusydi

NIM : 164221043

# Import Library

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity


# Import data

In [2]:
nltk.download('punkt')
nltk.download('stopwords')

df_obat = pd.read_csv('data_halodoc_.csv')
df_eval = pd.read_csv('dataset eval.csv')

df_obat

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,name,price,Aturan Pakai,Dosis,Efek Samping,Golongan Produk,Indikasi Umum,Kemasan,Komposisi,Kontra Indikasi,...,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77
0,Sildenafil Citrate 50 mg 4 Tablet,Rp96.500 - Rp168.500,Dikonsumsi satu jam sebelum melakukan aktivita...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Viagra 50 mg 1 Tablet,Rp156.800 - Rp164.900,Dikonsumsi satu jam sebelum melakukan aktivita...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Ericfil Tablet 100 mg,Rp77.100 - Rp104.000,Dikonsumsi satu jam sebelum melakukan aktivita...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Cialis Tablet 10 mg,Rp197.300 - Rp203.500,Diberikan sebelum atau setelah makan,PENGGUNAAN OBAT INI HARUS SESUAI DENGAN PETUNJ...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Ericfil 50 mg Odf,Rp36.500 - Rp44.400,Cara penggunaan Ericfil ODF: 1. Buka kemasan d...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4727,Optimum Nutrition Whey Gold Standard Vanilla 1...,Rp2.200.000,Dikonsumsi 30-60 menit seletah latihan dan ber...,Campurkan 1 scoop ON Whey dengan 1 gelas air (...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4728,Optimum Nutrition Whey Gold Standard Vanilla 5 Lb,Rp1.150.000,Dikonsumsi 30-60 menit seletah latihan dan ber...,Campurkan 1 scoop ON Whey dengan 1 gelas air (...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4729,Optimum Serious Mass Chocolate 12Lb,Rp1.100.000 - Rp1.100.100,Dikonsumsi diantara waktu makan atau kapanpun ...,Campurkan 2 scoop Serious Mass dengan 3 gelas ...,NaN,Pangan Olahan Diet Khusus,Membantu meningkatkan berat badan dengan kandu...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4730,Optimum Whey Gold Chocolate 10 Lb,Rp2.200.000,Dikonsumsi 30-60 menit seletah latihan dan ber...,Campurkan 1 scoop ON Whey dengan 1 gelas air (...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Data Preprocessing

In [3]:
def preprocess_text(text):
    if pd.isna(text):
        return ""
    text = str(text).lower()
    text = re.sub(r'[^\w\s]', '', text)
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('indonesian'))
    tokens = [token for token in tokens if token not in stop_words]
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(token) for token in tokens]
    return ' '.join(tokens)

columns = df_obat.columns
df_obat['dokumen'] = df_obat.apply(lambda x: ' '.join([str(x[col]) if pd.notna(x[col]) else '' for col in columns]), axis=1)

df_obat['processed_text'] = df_obat['dokumen'].apply(preprocess_text)

df_obat[['name', 'processed_text']]


,name,processed_text
0,Sildenafil Citrate 50 mg 4 Tablet,sildenafil citrat 50 mg 4 tablet rp96500 rp168...
1,Viagra 50 mg 1 Tablet,viagra 50 mg 1 tablet rp156800 rp164900 dikons...
2,Ericfil Tablet 100 mg,ericfil tablet 100 mg rp77100 rp104000 dikonsu...
3,Cialis Tablet 10 mg,ciali tablet 10 mg rp197300 rp203500 makan pen...
4,Ericfil 50 mg Odf,ericfil 50 mg odf rp36500 rp44400 penggunaan e...
...,...,...
4727,Optimum Nutrition Whey Gold Standard Vanilla 1...,optimum nutrit whey gold standard vanilla 10 l...
4728,Optimum Nutrition Whey Gold Standard Vanilla 5 Lb,optimum nutrit whey gold standard vanilla 5 lb...
4729,Optimum Serious Mass Chocolate 12Lb,optimum seriou mass chocol 12lb rp1100000 rp11...
4730,Optimum Whey Gold Chocolate 10 Lb,optimum whey gold chocol 10 lb rp2200000 dikon...


In [4]:
df_eval ['processed_question'] = df_eval['Pertanyaan'].apply(preprocess_text)
df_eval

,Pertanyaan,processed_question
0,Comtusi obat apa?,comtusi obat
1,balsem murah?,balsem murah
2,obat pereda nyeri gigi?,obat pereda nyeri gigi
3,Mengapa Boost Optium tidak cocok untuk orang d...,boost optium cocok orang dewasa gangguan medi
4,Mengapa penting untuk menggunakan SGM BBLR ata...,sgm bblr rekomendasi dokter
5,Bagaimana Sensi Dry Pants menjaga kesehatan ku...,sensi dri pant menjaga kesehatan kulit bayi


# Method 1: TF-IDF dengan Cosine Similarity

cara kerja :
1. pertanyaan dan dokumen diubah mennjadi TF-IDF
2. menghitung cosine similiarity antara dokumen dan pertanyaan
3. diambil 3 teratas yang memiliki skor relevansi tertinggi

In [5]:
def tfidf_search(query, documents):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([query] + documents)
    cosine_scores = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:])
    return cosine_scores[0]

all_results = []
for idx, row in df_eval.iterrows():
    query = row['processed_question']  
    documents = df_obat['processed_text'].to_list()  
    
    tfidf_scores = tfidf_search(query, documents)
    
    results = pd.DataFrame({
        'Nama_Obat': df_obat['name'],
        'Dokumen' : documents,
        'TF-IDF Score': tfidf_scores
    })
    
    results = results.sort_values(by='TF-IDF Score', ascending=False).head(3)  
    results['Query'] = row['Pertanyaan']
    all_results.append(results)

final_results = pd.concat(all_results, ignore_index=True)
final_results

,Nama_Obat,Dokumen,TF-IDF Score,Query
0,Comtusi Forte 10 Kapsul,comtusi fort 10 kapsul rp29100 rp32000 makan p...,0.238995,Comtusi obat apa?
1,Comtusi Sirup 60 ml,comtusi sirup 60 ml rp43000 rp55000 perut koso...,0.224736,Comtusi obat apa?
2,Comtusi Rasa Strawberry Syrup 60 ml,comtusi strawberri syrup 60 ml rp43200 rp52300...,0.145445,Comtusi obat apa?
3,Tresnojoyo Balsem Telon 40 g,tresnojoyo balsem telon 40 g rp16300 rp19700 h...,0.275424,balsem murah?
4,Balsem Tolak Angin 20 g,balsem tolak angin 20 g rp13400 rp18500 gosokk...,0.234173,balsem murah?
5,Balsem Otot Geliga Muscular Balm 20 g,balsem otot geliga muscular balm 20 g rp10200 ...,0.199931,balsem murah?
6,Paramex Nyeri Otot 4 Tablet,paramex nyeri otot 4 tablet rp1800 rp2500 dimi...,0.345048,obat pereda nyeri gigi?
7,Blood Kneeheat Sachet,blood kneeheat sachet rp21700 produk konsumen ...,0.322341,obat pereda nyeri gigi?
8,Colgate Toothpaste Advanced Fresh 150 g,colgat toothpast advanc fresh 150 g rp50500 rp...,0.311741,obat pereda nyeri gigi?
9,Boost Optimum 800 g,boost optimum 800 g rp234700 rp410400 larutkan...,0.262927,Mengapa Boost Optium tidak cocok untuk orang d...


dilihat dari hasil yang di dapat metode ini dapat dengan baik mencari dokumen terkait.

- pertanyaan 1 : semua dokumen relevan
- pertanyaan 2 : semua dokumen relevant
- pertanyaan 3 : 2 dokumen teratas memang merupakan obat nyeri namun bukan untuk gigi, sedangkan yang satunya memang tenetang gigi namun bukan untuk obt nyeri.
- pertanyaan 4 : semua dokumen relevant
- pertanyaan 5 : semua dokumen relevant
- pertanyaan 6 : semua dokumen relevant

# Method 2: BM25

cara kerja :
1. dukumen dan pertanyaan diubah menjadi TF-IDF
2. menghitung panjang dokumen dan rata-rata penjang dokumen
3. melakukan perhitungan skor BM25 untuk tiap-tiap dokumen, dengan mempertimbangkan frekuensi kata, IDF, panjang dokumen,dan parameter 𝑘1 dan b.
4. diambil 3 teratas dengan skor terbaik

In [6]:
def bm25_search(query, documents, k1=1.5, b=0.75):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(documents)
    query_vector = vectorizer.transform([query])
    
    doc_lengths = np.sum(tfidf_matrix.toarray(), axis=1)
    avg_doc_length = np.mean(doc_lengths)
    
    tfidf_array = tfidf_matrix.toarray()
    query_array = query_vector.toarray()[0]
    
    scores = []
    for i in range(len(documents)):
        score = 0
        for term_idx in range(len(query_array)):
            if query_array[term_idx] > 0:
                tf = tfidf_array[i, term_idx]
                idf = vectorizer.idf_[term_idx]
                numerator = tf * (k1 + 1)
                denominator = tf + k1 * (1 - b + b * doc_lengths[i] / avg_doc_length)
                score += idf * numerator / denominator
        scores.append(score)
    
    return np.array(scores)

all_results = []
for idx, row in df_eval.iterrows():
    query = row['processed_question'] 
    documents = df_obat['processed_text'].tolist()  
    
    bm25_scores = bm25_search(query, documents)
    
    results = pd.DataFrame({
        'Nama_Obat': df_obat['name'],
        'Dokumen' : documents,
        'BM25 Score': bm25_scores
    })
    
    results = results.sort_values(by='BM25 Score', ascending=False).head(3)  
    results['Query'] = row['Pertanyaan']  
    all_results.append(results)

final_results = pd.concat(all_results, ignore_index=True)

final_results


,Nama_Obat,Dokumen,BM25 Score,Query
0,Comtusi Sirup 60 ml,comtusi sirup 60 ml rp43000 rp55000 perut koso...,2.605608,Comtusi obat apa?
1,Comtusi Forte 10 Kapsul,comtusi fort 10 kapsul rp29100 rp32000 makan p...,1.918887,Comtusi obat apa?
2,Comtusi Rasa Strawberry Syrup 60 ml,comtusi strawberri syrup 60 ml rp43200 rp52300...,1.564920,Comtusi obat apa?
3,Balsem Tolak Angin 20 g,balsem tolak angin 20 g rp13400 rp18500 gosokk...,4.875784,balsem murah?
4,Tresnojoyo Balsem Telon 40 g,tresnojoyo balsem telon 40 g rp16300 rp19700 h...,4.824315,balsem murah?
5,Balsem Otot Geliga Muscular Balm 20 g,balsem otot geliga muscular balm 20 g rp10200 ...,3.985111,balsem murah?
6,Menstrucare,menstrucar rp22000 buka kemasan menstrucar per...,4.881508,obat pereda nyeri gigi?
7,Blood Kneeheat Sachet,blood kneeheat sachet rp21700 produk konsumen ...,4.546270,obat pereda nyeri gigi?
8,Paramex Nyeri Otot 4 Tablet,paramex nyeri otot 4 tablet rp1800 rp2500 dimi...,3.903851,obat pereda nyeri gigi?
9,Boost D 5000 IU 10 Tablet,boost d 5000 iu 10 tablet rp32600 minum obat s...,4.587316,Mengapa Boost Optium tidak cocok untuk orang d...


dari hasil yang didapatkan, dapat dilihat bahwa metode bisa cukup baik dalam mencari dokumen yang relevant
- pertanyaan 1 : semua dokumen relevan
- pertanyaan 2 : semua dokumen relevant
- pertanyaan 3 : semua dokumen teratas memang merupakan obat nyeri namun bukan untuk gigi
- pertanyaan 4 : 2 dokumen teratas memang obat dengan nama Boost nmun bukan yang ditanyakan, dokumen ke 3 relevant
- pertanyaan 5 : 2 dokumen teratas tidak relevant, dokumen ke 3 relavant
- pertanyaan 6 : semua dokumen relevant

# Method 3 : Jaccard Similarity

Cara kerja :

terdapat 2 tahap un tuk metode ini

1. menghitung Jaccard Similarity:
    -setiap kata dalam pertanyaan dan dokumen dipecah menjadi set untuk menghilangkan duplikasi.
    - Jccard similarity dihitung sebagai perbandingan elemen yang sama (intersection) dan total elemen yang unik (union).

2. mencari Jaccard Similarity untuk Dokumen:
    - fungsi jaccard_search menghitung kesamaan untuk setiap dokumen terhadap pertanyaan dan mengembalikan skor dalam bentuk array.

3. kemudian diambil dokumen dengan 3 scor terbesar untuk masing-masing pertanyaan

In [7]:
def jaccard_similarity(query, document):
    query_terms = set(query.split())
    doc_terms = set(document.split())
    
    intersection = len(query_terms.intersection(doc_terms))
    union = len(query_terms.union(doc_terms))
    
    return intersection / union if union > 0 else 0

def jaccard_search(query, documents):
    scores = []
    for doc in documents:
        score = jaccard_similarity(query, doc)
        scores.append(score)
    return np.array(scores)

all_results = []
for idx, row in df_eval.iterrows():
    query = row['processed_question']  
    documents = df_obat['processed_text'].tolist() 
    
    jaccard_scores = jaccard_search(query, documents)
    
    results = pd.DataFrame({
        'Nama_Obat': df_obat['name'],
        'Dokumen' : documents,
        'Jaccard Score': jaccard_scores
    })
    
    results = results.sort_values(by='Jaccard Score', ascending=False).head(3)  
    results['Query'] = row['Pertanyaan']  
    all_results.append(results)

final_results = pd.concat(all_results, ignore_index=True)

final_results

,Nama_Obat,Dokumen,Jaccard Score,Query
0,Duazat 4 Kaplet,duazat 4 kaplet rp69800 rp75800 obat kera mera...,0.100000,Comtusi obat apa?
1,Coyarin 6 Kaplet,coyarin 6 kaplet rp106700 rp111000 obat kera m...,0.100000,Comtusi obat apa?
2,Nosfocin Sirup 120 ml,nosfocin sirup 120 ml rp105500 obat kera merah du,0.100000,Comtusi obat apa?
3,Balsem Tolak Angin 20 g,balsem tolak angin 20 g rp13400 rp18500 gosokk...,0.100000,balsem murah?
4,Balsem Otot Geliga Muscular Balm 20 g,balsem otot geliga muscular balm 20 g rp10200 ...,0.071429,balsem murah?
5,Tresnojoyo Balsem Telon 40 g,tresnojoyo balsem telon 40 g rp16300 rp19700 h...,0.058824,balsem murah?
6,Menstrucare,menstrucar rp22000 buka kemasan menstrucar per...,0.222222,obat pereda nyeri gigi?
7,Pamol Sirup 60 ml,pamol sirup 60 ml rp20200 rp44000 makan anak 6...,0.142857,obat pereda nyeri gigi?
8,Orinox 120 mg 10 Tablet,orinox 120 mg 10 tablet rp162500 makan penggun...,0.111111,obat pereda nyeri gigi?
9,Decolsin 4 Kapsul,decolsin 4 kapsul rp3700 rp4300 dikonsumsi mak...,0.105263,Mengapa Boost Optium tidak cocok untuk orang d...


Setelah melakukan evaluasi, ditmukan bahwa masih banyak pertanyaan yang tidak mengarah ke dokumen obat yang relevan. Oleh karena itu, perlu dilakukan beberapa perbaikan dalam sistem pencarian.
- pertanyaan 1 : semua dokumen tidak relevan
- pertanyaan 2 : semua dokumen relevant
- pertanyaan 3 : pramol merupakan obat demam anak bukan obat nyeri, Menstrucare memang obat nyeri gigi namun bukan untuk obat nyeri gigi, sedangkan Orinox  merupakan memang obat nyeri gigi setelah oprasi.
- pertanyaan 4 : semua dokumen tidak relevant
- pertanyaan 5 : semua dokumen tidak relevant
- pertanyaan 6 : semua dokumen tidak relevant

# Method 4: LSA

Cara kerja : 
1. dokumen dan Pertanyaan diubah menjadi TF-IDF:
2. menggunakan Singular Value Decomposition (SVD):
    - matriks TF-IDF yang telah dibuat kemudian diolah dengan TruncatedSVD untuk mengurangi dimensi data. Proses ini menangkap struktur laten dalam data, sehingga mengurangi noise dan meningkatkan pemahaman semantik.
3. yransformasi Pertanyaan ke Ruang LSA:
4. menghitung Similarity
5. 3 dokumen dengan nilai similiaritu terbanyak disimpan


In [8]:
def lsa_search(query, documents, n_components=100):
    vectorizer = TfidfVectorizer()
    
    tfidf_matrix = vectorizer.fit_transform(documents)
    
    svd = TruncatedSVD(n_components=n_components)
    lsa_matrix = svd.fit_transform(tfidf_matrix)

    query_vector = vectorizer.transform([query])
    lsa_query = svd.transform(query_vector)

    cosine_scores = cosine_similarity(lsa_query, lsa_matrix).flatten()
    
    return cosine_scores

all_results_lsa = []
for idx, row in df_eval.iterrows():
    query = row['processed_question'] 
    documents = df_obat['processed_text'].tolist()  
    
    lsa_scores = lsa_search(query, documents)
    
    results = pd.DataFrame({
        'Nama_Obat': df_obat['name'],
        'Dokumen' : documents,
        'LSA Score': lsa_scores
    })
    
    results = results.sort_values(by='LSA Score', ascending=False).head(3)  
    results['Query'] = row['Pertanyaan']  
    all_results_lsa.append(results)

final_results_lsa = pd.concat(all_results_lsa, ignore_index=True)
final_results_lsa

,Nama_Obat,Dokumen,LSA Score,Query
0,Nebulizer Mask Adult,nebul mask adult rp43500 rp47900 1 masukkan ob...,0.859780,Comtusi obat apa?
1,Novagyl 500 mg 10 Tablet,novagyl 500 mg 10 tablet rp17300 rp17600 pengg...,0.785267,Comtusi obat apa?
2,Lantiflam Tablet 50 mg,lantiflam tablet 50 mg rp3800 rp4000 penggunaa...,0.755970,Comtusi obat apa?
3,Balsem Otot Geliga Muscular Balm 20 g,balsem otot geliga muscular balm 20 g rp10200 ...,0.709646,balsem murah?
4,Balsem Otot Geliga 20 g,balsem otot geliga 20 g rp9000 rp17200 diolesk...,0.702962,balsem murah?
5,Balsem Otot Geliga 40 g,balsem otot geliga 40 g rp18700 rp23700 dioles...,0.668355,balsem murah?
6,Paramex Nyeri Otot 4 Tablet,paramex nyeri otot 4 tablet rp1800 rp2500 dimi...,0.760717,obat pereda nyeri gigi?
7,Ketoprofen 100 mg 10 Tablet,ketoprofen 100 mg 10 tablet rp9300 rp19300 dik...,0.758938,obat pereda nyeri gigi?
8,Pronalges Suppositoria 100 mg,pronalg suppositoria 100 mg rp22200 rp24500 di...,0.705188,obat pereda nyeri gigi?
9,Boost Optimum 800 g,boost optimum 800 g rp234700 rp410400 larutkan...,0.686753,Mengapa Boost Optium tidak cocok untuk orang d...


Setelah melakukan evaluasi, ditmukan bahwa masih terdapt beberapa pertanyaan yang tidak mengarah ke dokumen obat yang relevan.

- pertanyaan 1 : semua dokumen tidak relevan
- pertanyaan 2 : semua dokumen relevant
- pertanyaan 3 : semua dokumen relevant
- pertanyaan 4 : semua dokumen relevant
- pertanyaan 5 : semua dokumen relevant
- pertanyaan 6 : semua dokumen relevant, namun 2 dokumne tertinggi memang relevant namun bukan merek yang di inginkan.

# Method 5: BoW

Cara kerja :
1. dukumen diubah menjadi matrik Bag of Words
2. menghitung vektor pertanyaan
3. menghitung Cosine similiarity
4. kemudiaan diambil 3 dokumen dingan nilai kemiripak yang paling tinggi


In [9]:
def bow_search(query, documents):
    vectorizer = CountVectorizer()
    count_matrix = vectorizer.fit_transform(documents)
    query_vector = vectorizer.transform([query])
    cosine_scores = cosine_similarity(query_vector, count_matrix).flatten()
    return cosine_scores

all_results_bow = []
for idx, row in df_eval.iterrows():
    query = row['processed_question'] 
    documents = df_obat['processed_text'].tolist()  
    
    bow_scores = bow_search(query, documents)
    
    results = pd.DataFrame({
        'Nama_Obat': df_obat['name'],
        'Dokumen': documents,
        'BOW Score': bow_scores
    })
    
    results = results.sort_values(by='BOW Score', ascending=False).head(3)
    results['Query'] = row['Pertanyaan']
    all_results_bow.append(results)

final_results_bow = pd.concat(all_results_bow, ignore_index=True)
final_results_bow

,Nama_Obat,Dokumen,BOW Score,Query
0,Novagyl 500 mg 10 Tablet,novagyl 500 mg 10 tablet rp17300 rp17600 pengg...,0.485071,Comtusi obat apa?
1,Nebulizer Mask Adult,nebul mask adult rp43500 rp47900 1 masukkan ob...,0.462910,Comtusi obat apa?
2,Cendo Vitrolenta Eye Drop 5 ml,cendo vitrolenta eye drop 5 ml rp42300 rp51400...,0.404061,Comtusi obat apa?
3,Tresnojoyo Balsem Telon 40 g,tresnojoyo balsem telon 40 g rp16300 rp19700 h...,0.436436,balsem murah?
4,Balsem Tolak Angin 20 g,balsem tolak angin 20 g rp13400 rp18500 gosokk...,0.353553,balsem murah?
5,Tresnojoyo Balsem Telon 20 g,tresnojoyo balsem telon 20 g rp9500 rp10900 hi...,0.300000,balsem murah?
6,Bimastan 500 mg 10 Kaplet,bimastan 500 mg 10 kaplet rp3900 rp6600 makan ...,0.476999,obat pereda nyeri gigi?
7,Argesid 500 mg 10 Kaplet,argesid 500 mg 10 kaplet rp14600 rp16400 makan...,0.461538,obat pereda nyeri gigi?
8,Ibuprofen 400 mg 10 Tablet,ibuprofen 400 mg 10 tablet rp3600 rp5300 makan...,0.461353,obat pereda nyeri gigi?
9,Pamperindo Adult Diapers M 8 Pieces,pamperindo adult diaper m 8 piec rp38500 rp578...,0.313911,Mengapa Boost Optium tidak cocok untuk orang d...


dilihat dari hasil yang didapat ternyata metode sudah cukup baik dalam mencari dokumen yang relevant dengan pertanyaan, namun masih tterdapat beberapa pertanyaan yang tidak mendapatkan dokumen obat yng relevan

- pertanyaan 1 : semua dokumen tidak relevan
- pertanyaan 2 : semua dokumen relevant
- pertanyaan 3 : semua dokumen relevan
- pertanyaan 4 : semua dokumen relevant
- pertanyaan 5 : semua dokumen relevant
- pertanyaan 6 : 2 dokumen relevant, namun dokumen ke 3 tidak relevant namun memang berkaitan dengan kulit.

# Evaluate

In [26]:
# Membaca keenam dataset
df1 = pd.read_csv('pertanyaan 1.csv')
df2 = pd.read_csv('pertanyaan 2.csv')
df3 = pd.read_csv('pertanyaan 3.csv')
df4 = pd.read_csv('pertanyaan 4.csv')
df5 = pd.read_csv('pertanyaan 5.csv')
df6 = pd.read_csv('pertanyaan 6.csv')

# Menggabungkan semua DataFrame menjadi satu
all_data = pd.concat([df1, df2, df3, df4, df5, df6], ignore_index=True)


In [27]:
# Fungsi untuk menghitung precision dan recall
def calculate_precision_recall(true_positive, false_positive, false_negative):
    precision = true_positive / (true_positive + false_positive) if (true_positive + false_positive) > 0 else 0
    recall = true_positive / (true_positive + false_negative) if (true_positive + false_negative) > 0 else 0
    return precision, recall

# Daftar untuk menyimpan hasil evaluasi
results = []

# Loop untuk setiap metode
methods = ['tfidf_search', 'bm25_search', 'jaccard_search', 'lsa_search', 'bow_search']

for method in methods:
    all_results = []  # Simpan hasil untuk semua pertanyaan

    # Loop untuk setiap pertanyaan di df_eval
    for idx, row in df_eval.iterrows():
        query = row['processed_question']
        documents = all_data['processed_text'].tolist()  # Ambil dokumen dari semua data
        
        # Hitung skor dengan metode yang sesuai
        if method == 'tfidf_search':
            scores = tfidf_search(query, documents)
        elif method == 'bm25_search':
            scores = bm25_search(query, documents)
        elif method == 'jaccard_search':
            scores = jaccard_search(query, documents)
        elif method == 'lsa_search':
            scores = lsa_search(query, documents)
        elif method == 'bow_search':
            scores = bow_search(query, documents)

        # Simpan hasil dalam DataFrame sementara
        results_df = pd.DataFrame({
            'Nama_Obat': all_data['name'],
            'Query': query,
            'Score': scores
        })

        # Ambil 3 hasil teratas
        top_results = results_df.sort_values(by='Score', ascending=False).head(3)
        all_results.append(top_results)

    # Gabungkan hasil untuk setiap metode
    final_results = pd.concat(all_results, ignore_index=True)

    # Hitung true positives, false positives, dan false negatives
    # Untuk menghitung ini, kamu perlu membandingkan hasil dengan jawaban yang benar.
    # Misalnya, kita asumsikan df_eval memiliki kolom 'Expected' yang menunjukkan nama obat yang benar.
    true_positive = len(final_results[final_results['Nama_Obat'].isin(df_eval['Expected'])])
    false_positive = len(final_results) - true_positive
    false_negative = len(df_eval['Expected']) - true_positive

    # Hitung precision dan recall
    precision, recall = calculate_precision_recall(true_positive, false_positive, false_negative)

    # Simpan hasil evaluasi untuk setiap metode
    results.append({
        'Method': method,
        'Precision': precision,
        'Recall': recall
    })

# Buat DataFrame hasil evaluasi
evaluation_results = pd.DataFrame(results)

# Tampilkan hasil evaluasi
print(evaluation_results)


KeyError: 'processed_text'